In [1]:
import random
from deap import base, creator, tools, algorithms
import pandas as pd
import numpy as np
from typing import Tuple
import xlsxwriter
# Dear Kuba if you don't have some module you can install it with %pip install xlsxwriter and run the cell.

In [2]:
MAX_EVAL: int = 10000
MAX_ALGO: int = 100
DOMAIN_R = (-30, 30)
DIMENSIONS: Tuple = (5)#, 15, 30)

In [3]:
# Evaluation functions
import numpy as np

def rosenbrock(x: np.ndarray) -> float:
    """
    The Generalized Rosenbrock function is a non-convex function used as a 
    performance test problem for optimization algorithms.
    It is also known as the "banana function" due to the shape of its contours.

    Parameters:
    x (np.ndarray): Input array of shape (n,).

    Returns:
    float: The value of the Generalized Rosenbrock function at the given point.
    """
    n = len(x)
    result = 0.0
    for i in range(n - 1):
        result += 100 * np.square(x[i + 1] - np.square(x[i])) + np.square(x[i] - 1)
    return result

def salomon(x: np.ndarray) -> float:
    """
    The Salomon function is commonly used as a benchmark for evaluating 
    optimization algorithms.
    It has a single global minimum at the origin and many local minima,
    which makes it a challenging problem for optimization.

    Parameters:
    x (np.ndarray): Input array of shape (n,).

    Returns:
    float: The value of the Salomon function at the given point.
    """
    norm = np.linalg.norm(x)
    result = 1 - np.cos(2 * np.pi * norm) + 0.1 * norm
    return result

def whitley(x: np.ndarray) -> float:
    """
    The Whitley function is a highly multimodal function often used to test the 
    robustness of optimization algorithms.
    It has numerous local minima, making it difficult to optimize.

    Parameters:
    x (np.ndarray): Input array of shape (n,).

    Returns:
    float: The value of the Whitley function at the given point.
    """
    n = len(x)
    result = 0.0
    for i in range(n):
        for j in range(n):
            term = 100 * np.square(x[j] - np.square(x[i])) + np.square(1 - x[i])
            result += (term ** 2) / 4000.0 - np.cos(term) + 1
    return result

# Rosenbrock's setup
- Binary repr (16 bits per dimention)
- Looking for minimum
- Selection: Tournament
- Crossover: Two-point crossover
- Mutation: Flip-bit
- Succession: Elitism


# TODO Salomon's setup

- Selection: SUS (Stochastic Universal Sampling) for better exploration.
- Crossover: Uniform crossover for diverse offspring.
- Mutation: Gaussian mutation for finer-grain adjustments.
- Succession: Elitism to preserve top performers.

# TODO Whitley's

- Selection: Rank selection to handle deceptive fitness peaks.
- Crossover: Two-point crossover for local exploitation.
- Mutation: Flip-bit mutation for exploration.
- Succession: Steady-state evolution to maintain diversity.

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Global minimization
creator.create("Individual", list, fitness=creator.FitnessMin)

In [5]:
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)  # Binary gene (0 or 1)
toolbox.register("evaluate", rosenbrock)
toolbox.register("mate", tools.cxTwoPoint)  # Two-point crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)  # Flip bit mutation
toolbox.register("select", tools.selTournament, tournsize=3) 

# This is test algo

def run_algorithm():
    population = toolbox.population(n=300)
    evaluations = 0

    run_data = []  # To store real values for this run

    while evaluations < MAX_EVAL:
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]

        # Evaluate fitness for invalid individuals
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)
            evaluations += 1

            # Decode binary to real values and store them
            real_values = [
                int("".join(map(str, ind[i * 16:(i + 1) * 16])), 2) / (2**16 - 1) * 60 - 30
                for i in range(5)
            ]
            run_data.extend(real_values)

            # Stop if max evaluations are reached
            if evaluations >= MAX_EVAL:
                break

        # Replace population with new generation
        population[:] = toolbox.select(offspring + population, k=len(population))

    return run_data

### Run the GA 100 times and collect data
raw_data = {f"Run_{i + 1}": [] for i in range(MAX_ALGO)}

for i in range(MAX_ALGO):
    print(f"Running GA for column {i + 1}...")
    raw_data[f"Run_{i + 1}"] = run_algorithm()

### Save results to Excel
df = pd.DataFrame(raw_data)
df.to_excel("rosenbrock_results_fixed.xlsx", index=False)

# This is actual algo

In [6]:
def run_algorithm_2(dimension, pop_size, cxpb, mutpb):
    # Register individual and population specific to the dimension
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=16 * dimension)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Initialize population
    population = toolbox.population(n=pop_size)
    evaluations = 0
    fitness_data = []  # Store fitness values for ECDF plotting
    local_max_eval: int = MAX_EVAL * dimension

    while evaluations < local_max_eval:
        # Generate offspring
        offspring = algorithms.varAnd(population, toolbox, cxpb=cxpb, mutpb=mutpb)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]

        # Evaluate invalid individuals
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)  # Evaluate fitness
            evaluations += 1

            # Append the fitness value to fitness_data
            fitness_data.append(ind.fitness.values[0])

            # Stop if the evaluation budget is exhausted
            if evaluations >= local_max_eval:
                break

        # Replace population with the new generation
        population[:] = toolbox.select(offspring + population, k=len(population))

    return fitness_data


In [7]:
writer = pd.ExcelWriter("rosenbrock_results.xlsx", engine='xlsxwriter')

#for dim in DIMENSIONS:
dim = 5
print(f"=== Starting GA for dimensions: {dim} ===")
pop_size = 100 if dim == 5 else (200 if dim == 15 else 400)
cxpb = 0.8
#mutpb = 0.05 if dim == 5 else (0.05 if dim == 15 else 0.05)
mutpb = 0.05

# Prepare raw data dictionary for storing runs
raw_data = {f"Run_{i + 1}": [] for i in range(MAX_ALGO)}

for i in range(MAX_ALGO):
    print(f"  Running experiment {i + 1} for dimension {dim}...")
    raw_data[f"Run_{i + 1}"] = run_algorithm_2(dim, pop_size, cxpb, mutpb)

# Convert to DataFrame and write to corresponding sheet
df = pd.DataFrame(raw_data)
df.to_excel(writer, sheet_name=f"rosenbrock_{dim}", index=False)

writer.close()
print("Results saved to rosenbrock_results.xlsx")


=== Starting GA for dimensions: 5 ===
  Running experiment 1 for dimension 5...


TypeError: object of type 'numpy.float64' has no len()